# Analyzing Column Completeness in a Pandas DataFrame
In this notebook, we will analyze the completeness of each column in a pandas DataFrame. Completeness is defined as the percentage of non-NA/null entries in each column. We will then export the column names along with their respective completeness percentages to a CSV file.


In [21]:
import pandas as pd
import os

input_path = '../shared/DataLibrary/inputs/Crew/MNP/CSV'
output_path = '../shared/DataLibrary/inputs/Crew/MNP/Analysis/column_completeness.csv'

## Top 5 Function
Function to get the top 5 most frequent values for each column


In [22]:
def get_top_5_frequent_values(column):
    return column.value_counts().head(5).index.tolist()


## Calculating Column Completeness and Top 5 Frequent Values
Here, we calculate the completeness of each column in the DataFrame, as well as identify the top 5 most frequent distinct values for each column. The completeness is calculated as the percentage of non-NA/null entries in each column.


In [23]:
def process_single_csv(df):
    data = []
    for col in df.columns:
        non_na_count = df[col].notna().sum()
        completeness_percentage = non_na_count / len(df) * 100
        top_5_values = get_top_5_frequent_values(df[col])
        data.append([col, non_na_count, completeness_percentage, top_5_values])

    completeness_df = pd.DataFrame(data, columns=['Column Name', 'Completeness (Absolute)', 'Completeness (%)', 'Top 5 Frequent Values'])
    completeness_df.sort_values('Completeness (%)', ascending=False, inplace=True)

    return completeness_df


## Function to process data from a directory of CSV files

In [24]:
def process_directory(input_dir, output_csv):
    all_dataframes = []
    base_schema = None
    for filename in os.listdir(input_dir):
        if filename.endswith(".csv"):
            input_csv = os.path.join(input_dir, filename)
            df = pd.read_csv(input_csv, low_memory=False)

            # Check if the schema of the current file matches the base schema
            if base_schema is None:
                base_schema = set(df.columns)
            elif set(df.columns) != base_schema:
                # Report mismatch and continue with the next file
                print(f"Schema mismatch in {filename}:")
                print("Columns in this file but not in base:", set(df.columns) - base_schema)
                print("Columns in base but not in this file:", base_schema - set(df.columns))
                continue

            all_dataframes.append(df)

    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        completeness_df = process_single_csv(combined_df)
        completeness_df.to_csv(output_csv, index=False)
    else:
        print("No files processed due to schema mismatches.")


## Creating a Completeness Report
In this section, we save the DataFrame containing column names, their completeness percentages, and the top 5 most frequent distinct values to a CSV file for further analysis.


In [25]:
if os.path.isfile(input_path):
    process_single_csv(input_path, output_path)
elif os.path.isdir(input_path):
    process_directory(input_path, output_path)
else:
    print("The input path is neither a file nor a directory.")

